In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
import seaborn as sns
from keras.layers import Input,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
import quandl
print(tf.__version__)

2.1.0


Using TensorFlow backend.


In [2]:
quandl.ApiConfig.api_key = '_66tvRgY5_szTgfB7aeB'
df = quandl.get('BITSTAMP/USD')
data = df
# data = pd.read_csv('bitcoin_data.csv')
data.reset_index()

,Date,High,Low,Last,Bid,Ask,Volume,VWAP
0,2014-04-15,515.00,453.16,499.01,500.01,505.04,28535.844106,491.41
1,2014-04-16,548.00,494.02,534.00,535.01,536.00,31159.941300,520.21
2,2014-04-17,537.24,481.63,506.52,504.70,505.38,21126.375080,504.83
3,2014-04-18,508.43,470.00,487.00,484.14,487.00,11879.484756,485.72
4,2014-04-19,507.43,472.81,504.74,504.74,505.00,10262.195861,492.22
...,...,...,...,...,...,...,...,...
2381,2020-11-17,16894.93,15875.50,16724.62,16719.84,16729.63,7511.143605,16409.99
2382,2020-11-18,17868.00,16570.00,17681.77,17670.67,17680.50,9824.899652,17260.21
2383,2020-11-19,18483.00,17041.47,17785.95,17783.71,17792.96,15717.759599,17874.21
2384,2020-11-20,18189.97,17346.62,17819.37,17817.05,17819.42,10507.652250,17792.16


In [3]:
# data = data.drop(columns='Date')


In [4]:
test = data.tail(20)
test

,High,Low,Last,Bid,Ask,Volume,VWAP
Date,,,,,,,
2020-11-01,13907.47,13629.31,13711.21,13700.59,13711.21,2465.795017,13766.72
2020-11-03,13842.50,13220.00,13563.72,13570.69,13578.98,7062.704712,13545.15
2020-11-04,14083.76,13287.70,14041.58,14030.89,14041.58,7226.516356,13633.01
2020-11-05,14277.50,13520.87,14160.59,14162.68,14171.73,10925.676805,13908.16
2020-11-06,15770.58,14100.00,15605.04,15603.49,15608.32,18422.631860,14937.25
2020-11-07,15968.98,15196.01,15598.09,15598.10,15603.23,13479.467068,15570.55
2020-11-08,15778.60,14351.00,14838.97,14832.01,14842.70,10933.924192,15076.99
2020-11-09,15664.90,14727.19,15489.15,15480.45,15490.09,5046.499164,15255.08
2020-11-10,15854.48,14824.66,15332.04,15335.50,15348.95,14466.079537,15364.34


In [5]:
High = test.High.values
Low = test.Low.values
Volume = test.Volume.values
Last = test.Last.values

In [6]:
X = []
y = []

for i in range(len(High)):
    row = []
    yrow = []
    row.append(High[i])
    row.append(Low[i])
#     row.append(Volume[i])
    yrow.append(Last[i])
    X.append(row)
    y.append(yrow)
X = np.array(X)
y = np.array(y)
X

array([[13907.47, 13629.31],
       [13842.5 , 13220.  ],
       [14083.76, 13287.7 ],
       [14277.5 , 13520.87],
       [15770.58, 14100.  ],
       [15968.98, 15196.01],
       [15778.6 , 14351.  ],
       [15664.9 , 14727.19],
       [15854.48, 14824.66],
       [15482.76, 15092.47],
       [15991.01, 15290.85],
       [16369.99, 15481.  ],
       [16494.52, 15970.33],
       [16341.89, 15715.1 ],
       [16170.  , 15786.46],
       [16894.93, 15875.5 ],
       [17868.  , 16570.  ],
       [18483.  , 17041.47],
       [18189.97, 17346.62],
       [18824.88, 17755.81]])

In [7]:


from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.preprocessing import StandardScaler




In [8]:
normalizer = preprocessing.Normalization()
normalizer.adapt(X_train)

In [9]:
horsepower = np.array(X_train)
horsepower2 = np.array(X_train)
horsepower_normalizer = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer.adapt(horsepower)
horsepower_normalizer2 = preprocessing.Normalization(input_shape=[1,])
horsepower_normalizer2.adapt(horsepower2)

In [10]:
# # input_layer = Input(shape=(X.shape[1],))
# dense_layer_1 = Dense(100, input_shape=(X.shape[1],), activation='relu')
# dense_layer_2 = Dense(50, activation='relu')(dense_layer_1)
# dense_layer_3 = Dense(25, activation='relu')(dense_layer_2)
# output = Dense(1)

# model = Model([
#     Dense(100, input_dim=X.shape[1], activation='relu'),
#     Dense(50, activation='relu'),
#     Dense(25, activation='relu'),
#     Dense(units=1, activation='softmax')
# ])
# model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

horsepower_model = tf.keras.Sequential([
#     horsepower_normalizer,
    layers.Dense(50, activation='relu', input_dim=2),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(25, activation='relu'),
    layers.Dense(units=1)
])


In [11]:
#horsepower_model.predict(X_test[:10])

In [12]:
horsepower_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001),
    loss='mean_absolute_error', metrics=['accuracy'])

In [13]:
%%time
history = horsepower_model.fit(
    X_train, y_train,
    epochs=700,
    # suppress logging
    verbose=2,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

Train on 12 samples, validate on 4 samples
Epoch 1/700
12/12 - 1s - loss: 15630.5947 - accuracy: 0.0000e+00 - val_loss: 14958.2188 - val_accuracy: 0.0000e+00
Epoch 2/700
12/12 - 0s - loss: 15581.3174 - accuracy: 0.0000e+00 - val_loss: 14910.9707 - val_accuracy: 0.0000e+00
Epoch 3/700
12/12 - 0s - loss: 15532.0986 - accuracy: 0.0000e+00 - val_loss: 14863.7793 - val_accuracy: 0.0000e+00
Epoch 4/700
12/12 - 0s - loss: 15482.9375 - accuracy: 0.0000e+00 - val_loss: 14814.5830 - val_accuracy: 0.0000e+00
Epoch 5/700
12/12 - 0s - loss: 15432.7998 - accuracy: 0.0000e+00 - val_loss: 14760.2051 - val_accuracy: 0.0000e+00
Epoch 6/700
12/12 - 0s - loss: 15377.1377 - accuracy: 0.0000e+00 - val_loss: 14704.7363 - val_accuracy: 0.0000e+00
Epoch 7/700
12/12 - 0s - loss: 15319.4092 - accuracy: 0.0000e+00 - val_loss: 14648.6641 - val_accuracy: 0.0000e+00
Epoch 8/700
12/12 - 0s - loss: 15261.0283 - accuracy: 0.0000e+00 - val_loss: 14592.1670 - val_accuracy: 0.0000e+00
Epoch 9/700
12/12 - 0s - loss: 15202.

Epoch 72/700
12/12 - 0s - loss: 10373.9854 - accuracy: 0.0000e+00 - val_loss: 9874.1592 - val_accuracy: 0.0000e+00
Epoch 73/700
12/12 - 0s - loss: 10269.5693 - accuracy: 0.0000e+00 - val_loss: 9772.8301 - val_accuracy: 0.0000e+00
Epoch 74/700
12/12 - 0s - loss: 10163.9717 - accuracy: 0.0000e+00 - val_loss: 9670.3906 - val_accuracy: 0.0000e+00
Epoch 75/700
12/12 - 0s - loss: 10057.3838 - accuracy: 0.0000e+00 - val_loss: 9566.5215 - val_accuracy: 0.0000e+00
Epoch 76/700
12/12 - 0s - loss: 9949.0469 - accuracy: 0.0000e+00 - val_loss: 9456.4160 - val_accuracy: 0.0000e+00
Epoch 77/700
12/12 - 0s - loss: 9834.3779 - accuracy: 0.0000e+00 - val_loss: 9344.1016 - val_accuracy: 0.0000e+00
Epoch 78/700
12/12 - 0s - loss: 9717.4521 - accuracy: 0.0000e+00 - val_loss: 9229.8066 - val_accuracy: 0.0000e+00
Epoch 79/700
12/12 - 0s - loss: 9598.3643 - accuracy: 0.0000e+00 - val_loss: 9113.6543 - val_accuracy: 0.0000e+00
Epoch 80/700
12/12 - 0s - loss: 9477.0791 - accuracy: 0.0000e+00 - val_loss: 8994.54

Epoch 144/700
12/12 - 0s - loss: 600.5637 - accuracy: 0.0000e+00 - val_loss: 499.3718 - val_accuracy: 0.0000e+00
Epoch 145/700
12/12 - 0s - loss: 556.8997 - accuracy: 0.0000e+00 - val_loss: 438.3625 - val_accuracy: 0.0000e+00
Epoch 146/700
12/12 - 0s - loss: 493.2773 - accuracy: 0.0000e+00 - val_loss: 360.6492 - val_accuracy: 0.0000e+00
Epoch 147/700
12/12 - 0s - loss: 412.2299 - accuracy: 0.0000e+00 - val_loss: 268.4250 - val_accuracy: 0.0000e+00
Epoch 148/700
12/12 - 0s - loss: 329.0136 - accuracy: 0.0000e+00 - val_loss: 166.9724 - val_accuracy: 0.0000e+00
Epoch 149/700
12/12 - 0s - loss: 243.9815 - accuracy: 0.0000e+00 - val_loss: 99.2380 - val_accuracy: 0.0000e+00
Epoch 150/700
12/12 - 0s - loss: 208.1087 - accuracy: 0.0000e+00 - val_loss: 99.8611 - val_accuracy: 0.0000e+00
Epoch 151/700
12/12 - 0s - loss: 189.2202 - accuracy: 0.0000e+00 - val_loss: 150.0007 - val_accuracy: 0.0000e+00
Epoch 152/700
12/12 - 0s - loss: 190.7557 - accuracy: 0.0000e+00 - val_loss: 205.9590 - val_accura

Epoch 217/700
12/12 - 0s - loss: 178.2939 - accuracy: 0.0000e+00 - val_loss: 127.9744 - val_accuracy: 0.0000e+00
Epoch 218/700
12/12 - 0s - loss: 178.5207 - accuracy: 0.0000e+00 - val_loss: 128.1318 - val_accuracy: 0.0000e+00
Epoch 219/700
12/12 - 0s - loss: 178.4588 - accuracy: 0.0000e+00 - val_loss: 130.7661 - val_accuracy: 0.0000e+00
Epoch 220/700
12/12 - 0s - loss: 178.3035 - accuracy: 0.0000e+00 - val_loss: 131.0747 - val_accuracy: 0.0000e+00
Epoch 221/700
12/12 - 0s - loss: 178.3933 - accuracy: 0.0000e+00 - val_loss: 129.2805 - val_accuracy: 0.0000e+00
Epoch 222/700
12/12 - 0s - loss: 178.3303 - accuracy: 0.0000e+00 - val_loss: 127.9338 - val_accuracy: 0.0000e+00
Epoch 223/700
12/12 - 0s - loss: 178.4952 - accuracy: 0.0000e+00 - val_loss: 129.2371 - val_accuracy: 0.0000e+00
Epoch 224/700
12/12 - 0s - loss: 178.3274 - accuracy: 0.0000e+00 - val_loss: 130.6880 - val_accuracy: 0.0000e+00
Epoch 225/700
12/12 - 0s - loss: 178.2833 - accuracy: 0.0000e+00 - val_loss: 129.9041 - val_accu

Epoch 290/700
12/12 - 0s - loss: 178.1681 - accuracy: 0.0000e+00 - val_loss: 131.9253 - val_accuracy: 0.0000e+00
Epoch 291/700
12/12 - 0s - loss: 178.6338 - accuracy: 0.0000e+00 - val_loss: 131.5549 - val_accuracy: 0.0000e+00
Epoch 292/700
12/12 - 0s - loss: 178.5270 - accuracy: 0.0000e+00 - val_loss: 128.8096 - val_accuracy: 0.0000e+00
Epoch 293/700
12/12 - 0s - loss: 178.1923 - accuracy: 0.0000e+00 - val_loss: 126.6494 - val_accuracy: 0.0000e+00
Epoch 294/700
12/12 - 0s - loss: 178.5172 - accuracy: 0.0000e+00 - val_loss: 127.6309 - val_accuracy: 0.0000e+00
Epoch 295/700
12/12 - 0s - loss: 178.2313 - accuracy: 0.0000e+00 - val_loss: 128.8325 - val_accuracy: 0.0000e+00
Epoch 296/700
12/12 - 0s - loss: 178.1823 - accuracy: 0.0000e+00 - val_loss: 130.2358 - val_accuracy: 0.0000e+00
Epoch 297/700
12/12 - 0s - loss: 178.1443 - accuracy: 0.0000e+00 - val_loss: 129.0662 - val_accuracy: 0.0000e+00
Epoch 298/700
12/12 - 0s - loss: 178.1665 - accuracy: 0.0000e+00 - val_loss: 128.3308 - val_accu

Epoch 363/700
12/12 - 0s - loss: 178.0369 - accuracy: 0.0000e+00 - val_loss: 128.3545 - val_accuracy: 0.0000e+00
Epoch 364/700
12/12 - 0s - loss: 178.0326 - accuracy: 0.0000e+00 - val_loss: 128.7649 - val_accuracy: 0.0000e+00
Epoch 365/700
12/12 - 0s - loss: 178.0146 - accuracy: 0.0000e+00 - val_loss: 129.4944 - val_accuracy: 0.0000e+00
Epoch 366/700
12/12 - 0s - loss: 177.9845 - accuracy: 0.0000e+00 - val_loss: 130.5103 - val_accuracy: 0.0000e+00
Epoch 367/700
12/12 - 0s - loss: 178.2037 - accuracy: 0.0000e+00 - val_loss: 128.6838 - val_accuracy: 0.0000e+00
Epoch 368/700
12/12 - 0s - loss: 178.0099 - accuracy: 0.0000e+00 - val_loss: 127.3992 - val_accuracy: 0.0000e+00
Epoch 369/700
12/12 - 0s - loss: 178.0559 - accuracy: 0.0000e+00 - val_loss: 126.6016 - val_accuracy: 0.0000e+00
Epoch 370/700
12/12 - 0s - loss: 178.1008 - accuracy: 0.0000e+00 - val_loss: 129.2146 - val_accuracy: 0.0000e+00
Epoch 371/700
12/12 - 0s - loss: 177.9820 - accuracy: 0.0000e+00 - val_loss: 131.9304 - val_accu

Epoch 436/700
12/12 - 0s - loss: 177.8737 - accuracy: 0.0000e+00 - val_loss: 127.9712 - val_accuracy: 0.0000e+00
Epoch 437/700
12/12 - 0s - loss: 177.8538 - accuracy: 0.0000e+00 - val_loss: 128.7778 - val_accuracy: 0.0000e+00
Epoch 438/700
12/12 - 0s - loss: 177.8206 - accuracy: 0.0000e+00 - val_loss: 129.9036 - val_accuracy: 0.0000e+00
Epoch 439/700
12/12 - 0s - loss: 177.9981 - accuracy: 0.0000e+00 - val_loss: 127.8796 - val_accuracy: 0.0000e+00
Epoch 440/700
12/12 - 0s - loss: 177.8495 - accuracy: 0.0000e+00 - val_loss: 126.4543 - val_accuracy: 0.0000e+00
Epoch 441/700
12/12 - 0s - loss: 177.9001 - accuracy: 0.0000e+00 - val_loss: 125.5693 - val_accuracy: 0.0000e+00
Epoch 442/700
12/12 - 0s - loss: 178.0235 - accuracy: 0.0000e+00 - val_loss: 128.4592 - val_accuracy: 0.0000e+00
Epoch 443/700
12/12 - 0s - loss: 177.8191 - accuracy: 0.0000e+00 - val_loss: 131.4644 - val_accuracy: 0.0000e+00
Epoch 444/700
12/12 - 0s - loss: 178.4447 - accuracy: 0.0000e+00 - val_loss: 131.1157 - val_accu

Epoch 509/700
12/12 - 0s - loss: 177.6812 - accuracy: 0.0000e+00 - val_loss: 127.8928 - val_accuracy: 0.0000e+00
Epoch 510/700
12/12 - 0s - loss: 177.6448 - accuracy: 0.0000e+00 - val_loss: 129.1226 - val_accuracy: 0.0000e+00
Epoch 511/700
12/12 - 0s - loss: 177.7392 - accuracy: 0.0000e+00 - val_loss: 126.9082 - val_accuracy: 0.0000e+00
Epoch 512/700
12/12 - 0s - loss: 177.6764 - accuracy: 0.0000e+00 - val_loss: 125.3501 - val_accuracy: 0.0000e+00
Epoch 513/700
12/12 - 0s - loss: 177.7318 - accuracy: 0.0000e+00 - val_loss: 124.3816 - val_accuracy: 0.0000e+00
Epoch 514/700
12/12 - 0s - loss: 177.9565 - accuracy: 0.0000e+00 - val_loss: 127.5376 - val_accuracy: 0.0000e+00
Epoch 515/700
12/12 - 0s - loss: 177.6433 - accuracy: 0.0000e+00 - val_loss: 130.8184 - val_accuracy: 0.0000e+00
Epoch 516/700
12/12 - 0s - loss: 178.2249 - accuracy: 0.0000e+00 - val_loss: 130.4360 - val_accuracy: 0.0000e+00
Epoch 517/700
12/12 - 0s - loss: 178.1147 - accuracy: 0.0000e+00 - val_loss: 126.7493 - val_accu

Epoch 582/700
12/12 - 0s - loss: 177.4588 - accuracy: 0.0000e+00 - val_loss: 128.1443 - val_accuracy: 0.0000e+00
Epoch 583/700
12/12 - 0s - loss: 177.4215 - accuracy: 0.0000e+00 - val_loss: 125.7485 - val_accuracy: 0.0000e+00
Epoch 584/700
12/12 - 0s - loss: 177.4923 - accuracy: 0.0000e+00 - val_loss: 124.0608 - val_accuracy: 0.0000e+00
Epoch 585/700
12/12 - 0s - loss: 177.5548 - accuracy: 0.0000e+00 - val_loss: 126.8906 - val_accuracy: 0.0000e+00
Epoch 586/700
12/12 - 0s - loss: 177.4429 - accuracy: 0.0000e+00 - val_loss: 129.9124 - val_accuracy: 0.0000e+00
Epoch 587/700
12/12 - 0s - loss: 177.9285 - accuracy: 0.0000e+00 - val_loss: 129.0293 - val_accuracy: 0.0000e+00
Epoch 588/700
12/12 - 0s - loss: 177.6738 - accuracy: 0.0000e+00 - val_loss: 124.6250 - val_accuracy: 0.0000e+00
Epoch 589/700
12/12 - 0s - loss: 177.5179 - accuracy: 0.0000e+00 - val_loss: 121.1289 - val_accuracy: 0.0000e+00
Epoch 590/700
12/12 - 0s - loss: 178.5358 - accuracy: 0.0000e+00 - val_loss: 122.3530 - val_accu

Epoch 655/700
12/12 - 0s - loss: 177.2475 - accuracy: 0.0000e+00 - val_loss: 125.1821 - val_accuracy: 0.0000e+00
Epoch 656/700
12/12 - 0s - loss: 177.2524 - accuracy: 0.0000e+00 - val_loss: 125.4880 - val_accuracy: 0.0000e+00
Epoch 657/700
12/12 - 0s - loss: 177.2367 - accuracy: 0.0000e+00 - val_loss: 126.2659 - val_accuracy: 0.0000e+00
Epoch 658/700
12/12 - 0s - loss: 177.2043 - accuracy: 0.0000e+00 - val_loss: 127.4688 - val_accuracy: 0.0000e+00
Epoch 659/700
12/12 - 0s - loss: 177.1840 - accuracy: 0.0000e+00 - val_loss: 124.7119 - val_accuracy: 0.0000e+00
Epoch 660/700
12/12 - 0s - loss: 177.2559 - accuracy: 0.0000e+00 - val_loss: 122.7302 - val_accuracy: 0.0000e+00
Epoch 661/700
12/12 - 0s - loss: 177.3870 - accuracy: 0.0000e+00 - val_loss: 125.5974 - val_accuracy: 0.0000e+00
Epoch 662/700
12/12 - 0s - loss: 177.2152 - accuracy: 0.0000e+00 - val_loss: 128.6851 - val_accuracy: 0.0000e+00
Epoch 663/700
12/12 - 0s - loss: 177.5312 - accuracy: 0.0000e+00 - val_loss: 127.6116 - val_accu

In [ ]:
%%time
# history = horsepower_model.fit(
#     X_train, y_train,
#     epochs=90,
#     # suppress logging
#     verbose=2,
#     # Calculate validation results on 20% of the training data
#     validation_split = 0.2)

In [22]:
import pickle
from tensorflow import keras

In [23]:
#pickle.dump(horsepower_model, open("model.pkl", "wb"))
horsepower_model.save("yellow")

INFO:tensorflow:Assets written to: yellow\assets


In [24]:
loaded_model = keras.models.load_model("yellow")

In [29]:
prediction = loaded_model.predict([[23, 24]])
print(prediction[0][0])

23.509787


In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
import pickle

In [ ]:
predict = horsepower_model.predict(X)
predict

In [ ]:
test_plt = test['Last'].reset_index()
plt.plot(test_plt, c='purple')
plt.plot(predict, c='g')

In [ ]:
#oSaver = tf.train.Saver()

#oSess = oSession
#oSaver.save(oSess, sModelPath)  #filename ends with .ckpt

pickle.dump(horsepower_model, open("model.pkl", "wb"))